## Identiface pipeline

In [ ]:
from io import BytesIO

import numpy as np
import pandas as pd
import tensorflow as tf
from huggingface_hub import hf_hub_download
from PIL import Image
from sklearn.model_selection import train_test_split


def download_dataset(n: int = 1000) -> pd.DataFrame:
    """Download facial image dataset from HuggingFace."""
    filepath = hf_hub_download(repo_id="rixmape/utkface", filename="data/train-00000-of-00001.parquet", repo_type="dataset")
    df = pd.read_parquet(filepath)
    return df.sample(n=n, random_state=42) if n > 0 else df


def prepare_data(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    """Process images and prepare training data."""
    images = []
    labels = []

    for _, row in df.iterrows():
        with Image.open(BytesIO(row["image"]["bytes"])) as img:
            img = img.convert("L").resize((48, 48))
            img_array = np.array(img, dtype=np.float32) / 255.0
            img_array = img_array.reshape(48, 48, 1)

            images.append(img_array)
            labels.append(row["gender"])

    return np.array(images), np.array(labels)


def build_vgg16() -> tf.keras.Sequential:
    """Build VGG16-like architecture using Sequential API."""
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1), name="block1_conv1"))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool"))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1"))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool"))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1"))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2"))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool"))

    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv1"))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv2"))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv3"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool"))

    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv1"))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv2"))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv3"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool"))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


def train_model(model: tf.keras.Sequential, X: np.ndarray, y: np.ndarray) -> tf.keras.Sequential:
    """Train model with class balancing and early stopping."""
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    class_counts = np.bincount(y_train)
    total = len(y_train)
    class_weights = {i: total / (len(class_counts) * count) for i, count in enumerate(class_counts)}
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64, class_weight=class_weights, callbacks=[early_stopping])

    return model


def run_pipeline() -> None:
    """Run the complete pipeline to train model and generate activation maps."""
    df = download_dataset(n=20000)
    X, y = prepare_data(df)
    model = build_vgg16()
    model = train_model(model, X, y)

    model.save("gender_classifier_vgg.h5")


if __name__ == "__main__":
    run_pipeline()

## BiasX pipeline

In [ ]:
from io import BytesIO

import numpy as np
import pandas as pd
import tensorflow as tf
from huggingface_hub import hf_hub_download
from keras.api.applications.vgg16 import VGG16
from keras.api.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.api.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.api.models import Model
from keras.api.optimizers import Adam
from keras.api.regularizers import l2
from keras.api.utils import to_categorical
from PIL import Image
from sklearn.model_selection import train_test_split


def download_dataset(n: int = 1000) -> pd.DataFrame:
    """Download facial image dataset from HuggingFace."""
    filepath = hf_hub_download(repo_id="rixmape/utkface", filename="data/train-00000-of-00001.parquet", repo_type="dataset")
    df = pd.read_parquet(filepath)
    return df.sample(n=n, random_state=42) if n > 0 else df


def prepare_images(df: pd.DataFrame) -> pd.DataFrame:
    """Process images to model-ready format."""

    def load_image(img_bytes):
        with Image.open(BytesIO(img_bytes)) as image:
            image = image.convert("L").resize((48, 48))
            image_array = np.array(image, dtype=np.float32) / 255.0
            return np.stack([image_array] * 3, axis=-1)

    df = df.copy()
    df["image_array"] = df.apply(lambda row: load_image(row["image"]["bytes"]), axis=1)
    return df


def split_data(df: pd.DataFrame) -> tuple:
    """Split data into train, validation and test sets."""
    images = np.stack(df["image_array"].values)
    labels = df["gender"].values

    X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

    y_train = to_categorical(y_train, num_classes=2)
    y_val = to_categorical(y_val, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    return X_train, X_val, X_test, y_train, y_val, y_test


def build_model() -> tf.keras.Model:
    """Build gender classification model."""
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    output = Dense(2, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"])

    return model


def train_model(model: tf.keras.Model, X_train: np.ndarray, y_train: np.ndarray, X_val: np.ndarray, y_val: np.ndarray) -> tf.keras.Model:
    """Train model with callbacks."""
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ModelCheckpoint("gender_classifier.keras", monitor="val_accuracy", save_best_only=True, mode="max"),
        ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
    ]

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, callbacks=callbacks)

    return model


def evaluate_model(model: tf.keras.Model, X_test: np.ndarray, y_test: np.ndarray) -> dict:
    """Evaluate model on test data."""
    return model.evaluate(X_test, y_test, return_dict=True)


def run_pipeline() -> None:
    """Run the complete training pipeline."""
    df = download_dataset()
    df = prepare_images(df)
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(df)

    model = build_model()
    model = train_model(model, X_train, y_train, X_val, y_val)

    results = evaluate_model(model, X_test, y_test)
    print(f"Test accuracy: {results['accuracy']:.4f}")


if __name__ == "__main__":
    run_pipeline()